# Projet de session : Phase 4 - Antoine Demers-Bergeron et Joseph Thompson


## Nouvelles fonctions implémentées

Pour commencer, nous avons implémenté différentes fonctions pour nous permettre de reconstruire les images.

Premièrement, nous avons vite remarqué que simplement utiliser l'algorithme de RSL ne serait pas suffisant. En effet, nos premiers tests en utilisant bêtement cet algorithme n'arrivaient tout simplement pas à reconstruire l'image. Nous avons donc implémenté une fonction qui applique l'algorithme de RSL sur un graphe avec l'algorithme de Prim en testant différents sommets de départ choisis au hasard et qui renvoie la meilleure solution trouvée. 

Nous avons décidé de tester sur un nombre restreints de sommets choisis au hasard simplement pour améliorer nos chances de trouver la meilleure solution. Itérer sur tous les sommets comme on le faisait dans la phase 3 était trop coûteux en temps. C'est pourquoi nous avons choisi cette approche.

In [ ]:
"""Teste (nbr_of_tests) sommets comme sommets de départ pour rsl avec l'algorithme de Prim et renvoie la meilleure solution trouvée"""
function semi_optimal_rsl(start_graph::Graph{Y,T}, nbr_of_tests::Int64) where {Y,T}
    optimal_cycle_length = Inf64
    optimal_cycle = 0
    optimal_method = "0"
    optimal_start_node = 0
    optimal_nodes_list = []
    
    nodes_to_try = rand(1:length(nodes(start_graph)), nbr_of_tests)
    for method_name in ["Prim"]
        println("Testing with ",method_name)
        
        for start_node_index in nodes_to_try
            println("Testing with node :", start_node_index)
            start_node = nodes(start_graph)[start_node_index]
            graph = deepcopy(start_graph)
            cycle, nodes_list = rsl(graph, start_node, method_name)
            value = sum_of_weights(cycle)
            
            if value < optimal_cycle_length
                
                optimal_cycle_length = value
                optimal_cycle = cycle
                optimal_method = method_name
                optimal_start_node = start_node
                optimal_nodes_list = nodes_list
            end

        end

    end
    println("La meilleure tournée que l'on peut trouver avec RSL est de longueur : ",optimal_cycle_length, "\nElle est obtenue à l'aide de l'algorithme de ",optimal_method," avec \
     ", optimal_start_node.name, " comme racine.")

    return optimal_cycle, optimal_nodes_list
end

Ces ajouts n'étaient pas suffisants pour bien reconstruire l'image, donc nous avons implémenté l'algorithme 2-opt pour améliorer la solution initiale trouvée. L'algorithme va effectuer des permutations dans l'ordre de la tournée de façon à ce que chaque permutation réduise le coût total de la tournée. On procède de la façon suivante : DÉCRIRE TWO_OPT

Avec ces outils ainsi que les fonctions fournies dans le fichier *tools.jl*, nous avons implémenté une fonction qui prend en entrée l'image mélangée ainsi que le fichier tsp correspondant et qui reconstruit l'image.

In [6]:
function shuffled_image_to_reconstruct(tsp_file::String,shuffled_image::String, new_image_name::String, nbr_of_tests::Int64 = 5)
    graph = graph_from_tsp(tsp_file,"graphe")[1]
    println("Calculating a tour")
   
    cycle, nodes_list = semi_optimal_rsl(graph, nbr_of_tests)   #Calcule une solution initiale
    println("rsl finished with cost ", sum_of_weights(cycle))
    tour_indexes = Vector{Int64}([])
    for node in nodes_list
        push!(tour_indexes, index(node))
    end
    println("running two opt")
    @time two_opt_cycle_graph, two_opt_cycle, two_opt_cost = run_two_opt(graph,cycle, tour_indexes, iter_stop = 3000) #Améliore la solution optimale
    
    one_index = findfirst(x -> x==1,two_opt_cycle)
    two_opt_cycle_mod = [two_opt_cycle[one_index:end] ; two_opt_cycle[1:one_index-1]]    #Réécrit la tournée avec le sommet 1 comme premier sommet
    
    nodes_index_list = two_opt_cycle_mod .- 1
    println("2 opt finished with cost ", sum_of_weights(two_opt_cycle_graph))
    
    
    tour = new_image_name * ".tour"
    write_tour(tour,nodes_index_list, sum_of_weights(two_opt_cycle_graph))
    
    reconstruct_picture(tour, shuffled_image, new_image_name * "-reconstruit.png",view = true)
end

shuffled_image_to_reconstruct (generic function with 2 methods)

## Tests

On peut tester cette dernières fonction sur différentes images. Voici les résultats : 

In [7]:
#shuffled_image_to_reconstruct("instances/tsp/instances/alaska-railroad.tsp","instances/images/shuffled/alaska-railroad.png","Train")

Pour alaska-railroad.png, on obtient une tournée de longueur 8105892 et les images suivantes

L'image originale  :
![title](../../instances/images/original/alaska-railroad.png)


Et l'image reconstruite : 
![title](../../Train-reconstruit.png)

